<a href="https://colab.research.google.com/github/rakshitrk/dog_breed/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)




cuda:0


In [2]:
# mounting google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd gdrive/My Drive

/content/gdrive/My Drive


In [0]:
# transforms the images of train, validation, test datasets
transform = {'train':transforms.Compose([transforms.Resize((244,244)),
                                      
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))]),
             
             'valid':transforms.Compose([transforms.Resize((244,244)),
                                     
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))]),
             
             'test':transforms.Compose([transforms.Resize((244,244)),
                                  
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
    
}

In [0]:
bs = 10
num_epochs = 22
# pytorch's ImageFolder to load the data
data = 'dogImages/'

train_data = datasets.ImageFolder(os.path.join(data, 'train/'), transform = transform['train']) 
val_data =  datasets.ImageFolder(os.path.join(data, 'train/'), transform = transform['valid']) 
test_data = datasets.ImageFolder(os.path.join(data, 'train/'), transform = transform['test']) 

train_loader = torch.utils.data.DataLoader(train_data, batch_size = bs, num_workers = 2, shuffle = True) 
val_loader = torch.utils.data.DataLoader(val_data, batch_size = bs, num_workers = 2, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = bs, num_workers = 2, shuffle = False)

In [0]:
# save breed names in text file
class_names = [item[4:].replace("_", " ") for item in train_loader.dataset.classes]

f = open("new.txt", "w")
for i in class_names:
  f.write(str(i) + "\n")
  
f.close()

In [4]:
# finetuning pretrained resnet152 model 
model = models.resnet152(pretrained = True)

ftrs = model.fc.in_features     # gives input dimentions of fullyconnected layer
model.fc = nn.Linear(ftrs,133)  # redesigning fully connected layer with 133 nodes(dog breeds)

model = model.to(device)
print(model)
  
  

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
241530880it [00:08, 28314277.68it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
# loss and optimizer for model
critersion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.002)

In [0]:
val_loss_min = np.Inf  # assigning minmum validation loss to infinity
  
for epoch in range(num_epochs):
    
  train_loss = 0.0  # running training loss
  val_loss = 0.0    # running validation loss
     
    
  # training model
  model.train()     # preparing model to train
  for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
      
    optimizer.zero_grad()
    output = model(inputs)
    loss = critersion(output, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*inputs.size(0)
      
      
  model.eval()     # preparing model for validation
  for inputs, labels in val_loader:
    inputs =  inputs.to(device)
    labels = labels.to(device)
    
    output = model( inputs)
    loss = critersion(output, labels)
    val_loss += loss.item()* inputs.size(0)
        
  train_loss = train_loss/len(train_loader.dataset)
  val_loss = val_loss/len(val_loader.dataset)
      
  print('Epoch:{}\nTraining Loss:{:6f}\tValidation Loss:{:6f}'.format(epoch,train_loss,val_loss))
      
  if val_loss <= val_loss_min:
    print('Validation Loss Dec. {:6f}--->{:6f}\t...SAVING...'.format(val_loss_min,val_loss))
    torch.save(model.state_dict(),'model.pt')         # saves model when validation loss dec. from min. validation loss
    val_loss_min = val_loss
      
  


Epoch:0
Training Loss:4.232339	Validation Loss:2.935760
Validation Loss Dec.    inf--->2.935760	...SAVING...
Epoch:1
Training Loss:2.757565	Validation Loss:1.484378
Validation Loss Dec. 2.935760--->1.484378	...SAVING...
Epoch:2
Training Loss:1.832774	Validation Loss:0.871715
Validation Loss Dec. 1.484378--->0.871715	...SAVING...
Epoch:3
Training Loss:1.316740	Validation Loss:0.564384
Validation Loss Dec. 0.871715--->0.564384	...SAVING...
Epoch:4
Training Loss:1.001743	Validation Loss:0.405004
Validation Loss Dec. 0.564384--->0.405004	...SAVING...
Epoch:5
Training Loss:0.792784	Validation Loss:0.297074
Validation Loss Dec. 0.405004--->0.297074	...SAVING...
Epoch:6
Training Loss:0.636654	Validation Loss:0.218471
Validation Loss Dec. 0.297074--->0.218471	...SAVING...
Epoch:7
Training Loss:0.519005	Validation Loss:0.161432
Validation Loss Dec. 0.218471--->0.161432	...SAVING...
Epoch:8
Training Loss:0.427331	Validation Loss:0.118752
Validation Loss Dec. 0.161432--->0.118752	...SAVING...
Epo

In [0]:
# loading trained model
model.load_state_dict(torch.load('model.pt'))

In [10]:
# accuracy of model on test data
correct = 0 # total no of correct predictions
total = 0   # total no of predictions / data

model.eval()  #preparing model for testing
for data in test_loader:
  
  images,labels = data
  images = images.to(device)
  labels = labels.to(device)
  
  output = model(images)
  _, pred = torch.max(output,1)
  
  total += labels.size(0)
  correct += (pred == labels).sum().item()
  
  acc = 100*correct/total
print('Accuracy of model:{} %'.format(acc))

Accuracy of model:99.91017964071857 %
